In [ ]:
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb#scrollTo=chW103JUItdk

# Using a pre-trained model and unfreezing some layers to make it specific to damage classification. 

In [ ]:
pip install tensorflow-gpu

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow-gpu as tf
keras = tf.keras

AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/riskmap_urop/none" 
!unzip -uq "/content/drive/My Drive/riskmap_urop/mild" 
!unzip -uq "/content/drive/My Drive/riskmap_urop/severe" 

In [ ]:
list_ds = tf.data.Dataset.list_files(str("all_*/*"))

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2] == class_names

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.cast(img, tf.float32)
  img = (img/127.5) - 1
  return tf.image.resize(img, [256, 256])

def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
class_names = ['all_none', 'all_mild', 'all_severe']

In [ ]:
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
print(labeled_ds)

<ParallelMapDataset shapes: ((256, 256, 3), (3,)), types: (tf.float32, tf.bool)>


In [ ]:
train_size = int(0.7 * 23272)
validation_size = int(0.15 * 23272)
test_size = int(0.15 * 23272)

train_dataset = labeled_ds.take(train_size)
validation_dataset =  labeled_ds.skip(train_size)
validation_dataset =  validation_dataset.take(validation_size)
test_dataset = labeled_ds.skip(validation_size)
test_dataset = test_dataset.take(test_size)

print(train_dataset)
print(validation_dataset)
print(test_dataset)

<TakeDataset shapes: ((256, 256, 3), (3,)), types: (tf.float32, tf.bool)>
<TakeDataset shapes: ((256, 256, 3), (3,)), types: (tf.float32, tf.bool)>
<TakeDataset shapes: ((256, 256, 3), (3,)), types: (tf.float32, tf.bool)>


In [ ]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
train = train_dataset.map(format_example)
validation = validation_dataset.map(format_example)
test = test_dataset.map(format_example)

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [ ]:
# inspect batch of data

for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

TensorShape([32, 160, 160, 3])

In [ ]:
# base model from the MobileNet V2
# pick which layer of MobileNet V2 you will use for feature extraction
# last classification layer isn't useful
# follow practice to depend on last layer before flatten operation (bottleneck layer)

In [ ]:
# instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet
# include_top=False argument means load network w/o classification layers at top
# ideal for feature extraction

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [ ]:
prediction_layer = tf.keras.layers.Dense(3) # logits = 3
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 3)


In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
len(model.trainable_variables)

2

In [ ]:
initial_epochs = 10
validation_steps = 20

loss0, accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

20/20 [==============================] - 15s 760ms/step - loss: 1.0342 - accuracy: 0.4406


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 1.03
initial accuracy: 0.44


In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

Epoch 1/10
Epoch 1/10
510/510 [==============================] - 676s 1s/step - loss: 0.9376 - accuracy: 0.5342 - val_loss: 0.9307 - val_accuracy: 0.5590
Epoch 2/10
Epoch 2/10
510/510 [==============================] - 670s 1s/step - loss: 0.9074 - accuracy: 0.5780 - val_loss: 0.9060 - val_accuracy: 0.5705
Epoch 3/10
Epoch 3/10
510/510 [==============================] - 688s 1s/step - loss: 0.8909 - accuracy: 0.6009 - val_loss: 0.8854 - val_accuracy: 0.6086
Epoch 4/10
Epoch 4/10
510/510 [==============================] - 699s 1s/step - loss: 0.8847 - accuracy: 0.6025 - val_loss: 0.8831 - val_accuracy: 0.6100
Epoch 5/10
Epoch 5/10
510/510 [==============================] - 709s 1s/step - loss: 0.8762 - accuracy: 0.6090 - val_loss: 0.8769 - val_accuracy: 0.6037
Epoch 6/10
Epoch 6/10
510/510 [==============================] - 721s 1s/step - loss: 0.8652 - accuracy: 0.6206 - val_loss: 0.8694 - val_accuracy: 0.6241
Epoch 7/10
Epoch 7/10
510/510 [==============================] - 719s 1s/ste

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.save("damagemodel.hdf5")
!cp earthquakemodel.hdf5 drive/My\ Drive/riskmap_urop_data